In [1]:
!pip show pandas

Name: pandas
Version: 2.0.1
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: 
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License
        
        Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
        All rights reserved.
        
        Copyright (c) 2011-2023, Open source contributors.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are met:
        
        * Redistributions of source code must retain the above copyright notice, this
          list of conditions and the following disclaimer.
        
        * Redistributions in binary form must reproduce the above copyright notice,
          this list of conditions and the following disclaimer in the documentation
          and/or other materials provided with the distrib

In [2]:
pip install tabulate matplotlib pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [17]:
# Importing the required modules.
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.pyplot import figure
from tabulate import tabulate

# Read data.
df = pd.read_csv(f'wf_security_executions.csv', header='infer')
# alpha_time,alpha_Cost,alpha_security

# print(df[df['instance'].str.startswith('Synthetic_')])
# type(df['instance'])

toy_instances = [
    "Synthetic_005_A", 
    "Synthetic_005_B", 
    "Synthetic_005_C", 
    "Synthetic_010_A", 
    "Synthetic_010_B", 
    "Synthetic_010_C",
    "Synthetic_015_A", 
    "Synthetic_015_B", 
    "Synthetic_015_C"
]

instances = [
    "CyberShake_030", 
    "CyberShake_050", 
    "CyberShake_100", 
#    "Epigenomics_024", 
#    "Epigenomics_046", 
#    "Epigenomics_100",
#    "Inspiral_030", 
#    "Inspiral_050", 
#    "Inspiral_100",
    "Montage_025", 
    "Montage_050", 
    "Montage_100",
#    "Sipht_030", 
#    "Sipht_060", 
#    "Sipht_100"
]

scenarios = [
#    1,
#    2,
#    3,
    4
]

# print(newdf)


# dir(df)
# print(df)

algorithms = [
    'grch',
    'heft',
    'grasp'
]

for instance in instances:
    print('Início da instância', instance)
    for scenario in scenarios:
        print('Início do scenario', scenario)
        for algorithm in algorithms:
            newdf = df[(1==1)
                       & (df['algorithm'] == algorithm)
                       # & (df.alpha_time == 0.90) 
                       # & (df.alpha_cost == .05) 
                       # & (df.alpha_security == .05) 
                       & (df['instance'].str.startswith(instance))
                       & (df.scenario == scenario)
                       & (df.num_buckets == 16)
                      ]
            print(tabulate(newdf.groupby(['instance', 'scenario', 'num_vm', 'num_buckets', 'alpha_time', 'alpha_cost', 
                                          'alpha_security', 'algorithm',  'num_clouds', 
                                          'num_crypt_requirements', 'num_conf_requirements', 'cloud_type', 
                                          'num_iterations', 'num_allocation_experiments']).mean()))
        print('Fim do scenario', scenario)
    print('Fim da instância', instance)
    print()
print('FIM')

# print(data.groupby(by='Algorithm').sum())
# data.groupby(['Algorithm', 'Instance', 'Scenario', 'Time alpha', 'Cost alpha', 'Security alpha'])

# print(data)
# print(tabulate(data))

# x axis values
# x = [1,2,3]
# corresponding y axis values
# y = [2,4,1]
  
# plotting the points 
# plt.plot(x, y)
  
# naming the x axis
# plt.xlabel('x - axis')
# naming the y axis
# plt.ylabel('y - axis')
  
# giving a title to my graph
# plt.title('My first graph!')
  
# function to show the plot
# plt.show()

Início da instância CyberShake_030
Início do scenario 4
---------------------------------------------------------------------------------------  --------  ---  -----  --------  --------
('CyberShake_030', 4, 1, 16, 0.05, 0.05, 0.9, 'grch', 4, 2, 4, 'heterogeneous', 100, 4)  0.110227  171  76.34  0.077519  0.479074
('CyberShake_030', 4, 1, 16, 0.05, 0.9, 0.05, 'grch', 4, 2, 4, 'heterogeneous', 100, 4)  0.302707  351  56.14  0.124031  0.472922
('CyberShake_030', 4, 1, 16, 0.9, 0.05, 0.05, 'grch', 4, 2, 4, 'heterogeneous', 100, 4)  0.348354  143  83.16  0.116279  0.450786
---------------------------------------------------------------------------------------  --------  ---  -----  --------  --------
---------------------------------------------------------------------------------------  --------  ---  -----  --------  --------
('CyberShake_030', 4, 1, 16, 0.05, 0.05, 0.9, 'heft', 4, 2, 4, 'heterogeneous', 100, 4)  0.275669  141  83.34  0.263566  0.001018
('CyberShake_030', 4, 1, 16, 0.05,